In [57]:
import os, sys, pytz
import pandas as pd
from django.apps import apps as django_apps
from django.db import DEFAULT_DB_ALIAS, connections
from django.db.migrations.loader import MigrationLoader
from django.db.migrations.recorder import MigrationRecorder
from django.core.exceptions import FieldDoesNotExist
from edc_base.model_mixins import ListModelMixin
from edc_visit_schedule.model_mixins import OnScheduleModelMixin

sys.path.append('../') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

tz = pytz.timezone('Africa/Gaborone')
connection = connections[DEFAULT_DB_ALIAS]
loader = MigrationLoader(connection)

In [58]:
m2m_models = []
for app_model in list(django_apps.get_app_configs()):
    models = app_model.models
    if models and 'flourish' in app_model.label:
        for model_label, model_cls in models.items():
            if (issubclass(model_cls, (ListModelMixin, OnScheduleModelMixin)) or
                'historical' in model_label or 'onschedule' in model_label):
                m2m_models.append(model_label)

In [59]:
migrations = MigrationRecorder.Migration.objects.filter(app__startswith='flourish')

In [60]:
exclude_fields = ['created', '_state', 'hostname_created', 'hostname_modified',
                  'revision', 'device_created', 'device_modified', 'id', 'site_id',
                  'created_time', 'modified_time', 'report_datetime_time',
                  'registration_datetime_time', 'screening_datetime_time', 'modified',
                  'form_as_json', 'consent_model', 'randomization_datetime',
                  'registration_datetime', 'is_verified_datetime', 'first_name',
                  'last_name', 'initials', 'guardian_name', 'identity', 'infant_visit_id',
                  'maternal_visit_id', 'processed', 'processed_datetime', 'packed',
                  'packed_datetime', 'shipped', 'shipped_datetime', 'received_datetime',
                  'identifier_prefix', 'primary_aliquot_identifier', 'clinic_verified',
                  'clinic_verified_datetime', 'drawn_datetime', 'slug', 'confirm_identity',
                  'related_tracking_identifier', 'parent_tracking_identifier', 'site',
                  'subject_consent_id', '_django_version', 'consent_identifier',
                  'subject_identifier_as_pk', 'user_created', 'user_modified', 'is_verified']

In [61]:
records = []
fields_verbose = {}
for migration in migrations:
    try:
        operations = loader.get_migration_by_prefix(migration.app, migration.name).operations
    except KeyError:
        continue
    else:
        for operation in operations:
            operation_type, _, details = operation.deconstruct()
            dt_applied = migration.applied.astimezone(tz)
            record = {'date_applied': dt_applied.strftime('%d-%m-%Y %H:%M %p'), }

            model_name = details.get('name', None) or details.get('model_name', None)

            # Exclude list_models
            if model_name and model_name.lower() in m2m_models:
                continue

            if operation_type == 'CreateModel':
                operation_applied = operation.describe().replace('model', 'form')
                fields = details.get('fields')
                field_list = []

                for field in fields:
                    field_name = field[0]
                    if field_name not in exclude_fields:
                        try:
                            field_verbose = field[1].verbose_name
                        except FieldDoesNotExist:
                            continue
                        else:
                            if field_verbose:
                                field_list.append(field_verbose)
                                fields_verbose.update({f'{field_name}': field_verbose})

                record.update(
                    {'operation_applied': operation_applied,
                     'form_name': model_name,
                     'field(s)': '\n'.join(field_list) })
            elif operation_type == 'RenameModel':
                operation_applied = operation.describe().replace('model', 'form')
                record.update({
                    'operation_applied': operation_applied,
                    'form_name': f'old_name: {details.get("old_name")} -> new_name: {details.get("new_name")}',
                    'field(s)': ''
                })
            elif operation_type in ['RemoveField', 'AddField', 'AlterField']:
                field_name = details.get('name')
                if field_name in exclude_fields:
                    continue
                field = details.get('field', None)
                field_verbose = getattr(field, 'verbose_name', fields_verbose.get(field_name, ''))
                operation_applied = operation.describe().replace(field_name, f'"{field_verbose}"')

                record.update({
                    'operation_applied': operation_applied,
                    'form_name': model_name,
                    'field(s)': field_verbose
                })
            elif operation_type == 'DeleteModel':
                operation_applied = operation.describe().replace('model', 'form')

                record.update({
                    'operation_applied': operation_applied,
                    'form_name': model_name,
                    'field(s)': ''
                })

            elif operation_type in ['AlterModelManagers', 'AlterModelOptions', 'RunPython',
                                    'AlterUniqueTogether', 'RenameField']:
                continue
            else:
                print(operation_type, details, migration.applied, operation.describe(), operation.deconstruct(), )
                print('\n')
            records.append(record)

In [62]:
records

[{'date_applied': '24-04-2021 05:04 AM',
  'operation_applied': 'Create form AntenatalEnrollment',
  'form_name': 'AntenatalEnrollment',
  'field(s)': "Subject Identifier\nReport date\nAre you diabetic?\nDo you intent to breast-feed your child for 6 months?\nWhat is your current HIV status?\n(Interviewer) Have you seen evidence of the HIV result?\nHave you tested for HIV before or during this pregnancy?\nDate of HIV Test\nWhat was your result?\n(Interviewer) Have you seen evidence of the result from HIV test on or before this pregnancy?\n(Interviewer) If HIV+ve, do records show that participant is taking, is prescribed,or will be prescribed ARVs (if newly diagnosed) during pregnancy?\nWas a rapid test processed?\nDate of rapid test\nWhat is the rapid test result?\nReason not enrolled\nDoes the mother know the approximate date of the first day her last menstrual period?\nWhat is the approximate date of the first day of the mother’s last menstrual period\nGA by LMP at enrollment.\nWhat i

In [63]:
df = pd.DataFrame(records)
# df.to_csv('migrations_details.csv', encoding='utf-8', index=False)

In [64]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('migrations_details.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='form_revisions', index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['form_revisions']

# Add a text wrap format.
text_wrap_format = workbook.add_format({'text_wrap': True})

# Add the format to column 2 (zero-indexed) and adjust the width.
worksheet.set_column(1, 3, 15, text_wrap_format)

# Close the Pandas Excel writer and output the Excel file.
writer.close()